In [9]:
import numpy as np
import pandas as pd

In [10]:
train = pd.read_csv(r'E:\Santander Recommendation\train_ver2.csv')

C:\Users\liamc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
train = train.loc[train['fecha_dato']=='2016-05-28']

In [12]:
product_cols = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1',
                'ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1',
                'ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_dela_fin_ult1','ind_deme_fin_ult1',
                'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_nom_pens_ult1',
                'ind_nomina_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
                'ind_recibo_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1']

In [13]:
test = pd.read_csv(r'E:\Santander Recommendation\test_ver2.csv')

C:\Users\liamc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
test_ids = pd.DataFrame(test['ncodpers'])

In [15]:
labels = train[product_cols]

In [16]:
labels['ncodpers'] = train.ncodpers

C:\Users\liamc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [17]:
labels = labels.set_index("ncodpers")

In [18]:
stacked_labels = labels.stack()

In [19]:
filtered_labels = stacked_labels.reset_index()

In [20]:
filtered_labels.columns = ["ncodpers", "product", "interaction"]

In [21]:
filtered_labels['product'] = filtered_labels['product'].astype('category').cat.codes

In [22]:
filtered_labels.head()

,ncodpers,product,interaction
0,657640,0,0.0
1,657640,1,0.0
2,657640,2,0.0
3,657640,3,0.0
4,657640,4,0.0


In [23]:
interactions = filtered_labels.loc[filtered_labels['interaction']==1]

In [26]:
interactions_ids = pd.DataFrame(interactions.ncodpers.drop_duplicates())

In [27]:
model_ids = pd.merge(interactions_ids,test_ids,how='inner',on='ncodpers')

In [30]:
labels = labels.reset_index()

In [31]:
labels = labels.merge(model_ids,how='inner',on='ncodpers')

In [34]:
labels = labels.set_index("ncodpers")

In [35]:
stacked_labels = labels.stack()

In [36]:
filtered_labels = stacked_labels.reset_index()

In [37]:
filtered_labels.columns = ["ncodpers", "product", "interaction"]

In [38]:
filtered_labels['product'] = filtered_labels['product'].astype('category').cat.codes

In [39]:
filtered_labels.head()

,ncodpers,product,interaction
0,657788,0,0.0
1,657788,1,0.0
2,657788,2,1.0
3,657788,3,0.0
4,657788,4,0.0


In [40]:
interactions = filtered_labels.loc[filtered_labels['interaction']==1]

In [41]:
interactions['user_emb_id'] = interactions['ncodpers'].astype('category').cat.codes

C:\Users\liamc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [42]:
joindf = interactions[['ncodpers','user_emb_id']].drop_duplicates()

In [43]:
test_usersdf = joindf.merge(test_ids,how='inner',on='ncodpers')

In [44]:
test_usersdf

,ncodpers,user_emb_id
0,657788,230626
1,657795,230630
2,657790,230628
3,657794,230629
4,657789,230627
5,657787,230625
6,657777,230620
7,657781,230623
8,657780,230622
9,657779,230621


In [45]:
test_users = test_usersdf['user_emb_id'].values

In [46]:
print(interactions.ncodpers.nunique())
print(interactions.user_emb_id.max())

696467
696466


In [47]:
user_ids = interactions['user_emb_id'].values
items_ids = interactions['product'].values

In [48]:
from spotlight.interactions import Interactions

In [49]:
interactionmatrix = Interactions(user_ids,items_ids)

In [50]:
from spotlight.cross_validation import random_train_test_split
from spotlight.evaluation import mrr_score
from spotlight.factorization.implicit import ImplicitFactorizationModel

In [51]:
train, test = random_train_test_split(interactionmatrix)

In [52]:
model = ImplicitFactorizationModel(n_iter=3,loss='bpr',use_cuda=True)
model.fit(train,verbose=True)

Epoch 0: loss 0.16094491217042295
Epoch 1: loss 0.11496981945458626
Epoch 2: loss 0.09522254034965223


In [30]:
#mrr = mrr_score(model, test)

In [53]:
pred = model.predict(test_users)

RuntimeError: The expanded size of the tensor (24) must match the existing size (696467) at non-singleton dimension 0

In [ ]:
#Here I can add in the part starting from the beginning in sample submission but just do the 7 most common items.

In [4]:
import torch

In [8]:
torch.cuda.is_available()

True